# 0. Import packages and load data

In [2]:
# Import necessary packages for the project
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy.stats as stats
import pylab as pl
import statistics

## import necessary classes and functions for the regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
!pip install utils
from utils import *
from sklearn.metrics import r2_score



  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=768ffebad9b2bc9e88c554848b1828c601b570a54312f1d0927bcc97a7335886
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [3]:
#Load data

main_data = pickle.load(open("python_database.p","rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'python_database.p'

In [ ]:
#Print data

#main_data.head(20)

# 1. Data exploration

## 1.1 Test plot on id 1001.1

In [ ]:
#Load oct_data of id 1001.1

oct_data_1 = main_data.iloc[0].oct_data
#oct_data_1

In [ ]:
#plt.figure(1) #Open new figure
#plt.scatter(oct_data_1.index, oct_data_1.depth_measured.values, color = 'b', marker = '.', label = 'Messpunkte') #Plot measurements
#plt.hlines(main_data.iloc[0].depth,0, oct_data_1.index[-1], color = 'r', label = 'Einschweißtiefe') #Plot horizontal line for weld depth
#plt.hlines(oct_data_1.depth_measured.values.mean(),0, oct_data_1.index[-1], color = 'y', label = 'Mittelwert Messpunkte') #Plot horizontal line for mean of measured depth by oct
#plt.plot(oct_data_1.depth_measured.rolling(500).mean(), color = 'g',label = 'Messungen gemittelt über 500 Messpunkte') #Plot mean value for every 500 points
#plt.title("id " + str(main_data.index[0]))
#plt.ylabel ("Tiefe [µm]")
#plt.xlabel ("Zeit in [s]")
#plt.legend()

## 1.2 Plot for all test series

In [ ]:
#Plots for unfiltered data

#rolling_value = 500 #Enter rolling value#
#for i in range(10,20): #Enter integer in range 0 and 630

   # plt.figure(i) #Open new figure
   # plt.scatter(main_data.iloc[i].oct_data.index, main_data.iloc[i].oct_data.depth_measured.values, color = 'b', marker = '.', label = 'Messpunkte') #Plot measurements
   # plt.hlines(main_data.iloc[i].depth,0, main_data.iloc[i].oct_data.index[-1], color = 'r', label = 'Einschweißtiefe') #Plot horizontal line for weld depth
   # plt.hlines(main_data.iloc[i].oct_data.depth_measured.values.mean(),0, main_data.iloc[i].oct_data.index[-1], color = 'y', label = 'Mittelwert Messpunkte') #Plot horizontal line for mean of measured depth by oct
   # plt.plot(main_data.iloc[i].oct_data.depth_measured.rolling(rolling_value).mean(), color = 'g',label = 'Messungen gemittelt über ' + str(rolling_value) + ' Messpunkte') #Plot mean value for every 500 points
   # plt.title('id ' + str(main_data.index[i]))
   # plt.ylabel ("Tiefe [µm]")
   # plt.xlabel ("Zeit in [s]")
   # plt.legend()

## 1.3 Lines with the same data sets

In [ ]:
#Print array with line numbers of data with the same parameters

same_param = []
same_param_list = [] #List for line numbers of data with same parameters
incl_param = []
incl_param_list = [] #List for already used parameter sets

for i in range (0, len(main_data)):
        if [main_data.iloc[i].power, main_data.iloc[i].speed, main_data.iloc[i].angle] not in incl_param_list:
            same_param = list(np.where((main_data['power'] == main_data.iloc[i].power) & (main_data['speed'] == main_data.iloc[i].speed) & (main_data['angle'] == main_data.iloc[i].angle)))
            incl_param = [main_data.iloc[i].power, main_data.iloc[i].speed, main_data.iloc[i].angle]
            same_param_list.append(same_param)
            incl_param_list.append(incl_param)


#print(incl_param_list)
#print(same_param_list)

## 1.4 Test on missing data

In [ ]:
#Test on missing data in column "power", "speed", "angle"
print ('In column "power" no entry is missing: ' + str(np.any(main_data['power'])))
print ('In column "speed" no entry is missing: ' + str(np.any(main_data['speed'])))
print ('In column "angle" no entry is missing: ' + str(np.any(main_data['angle'])))

# 2. Data preparation

In [ ]:
#Histogram of unfiltered data of id 1001.1

h = sorted(main_data.iloc[0].oct_data.depth_measured.values)  #sorted

fit = stats.norm.pdf(h, np.mean(h), np.std(h))  #this is a fitting indeed

pl.plot(h,fit,'-')

pl.hist(h,density=True)      #histogram of data

pl.axis( [0, 1500, 0, 0.004] )
pl.xlabel ("gemessene Tiefe in [µm]")
pl.ylabel ("Wahrscheinlichkeitsdichte")
pl.title ('id 1001.1 aus unfiltered_data')

pl.show()

#print(np.std(h))

## 2.1 Control for the test and the plots

In [ ]:
#Test definitions
rolling_val=1000
quantil_low =0.05
quantil_max =0.95
scaler = StandardScaler()
Test = "T1"


# Plots characteristics
plot_format = '.jpeg'

ts_size = 0.33 # the size of the test

plot_title_1 = "Variation des Polynom-Grades"

plot_title_2 = "Bayesian-Ridge-Regression"

plot_title_3 = "OLS linear regression"

## 2.2 Filtering of data in size of the moving window and percentile

In [ ]:
#Filtering of data in size of the moving window and percentile

depth_5_95 = []

for idx, row in main_data.iterrows():

    percentil_5 = row.oct_data.depth_measured.rolling(rolling_val).quantile(quantil_low) #rolling and quantile can be adapted
    percentil_95 = row.oct_data.depth_measured.rolling(rolling_val).quantile(quantil_max) #rolling and quantile can be adapted
    depth_5_95.append(row["oct_data"][lambda x: x.depth_measured.between(percentil_5, percentil_95)])


In [ ]:
#Adding filtered data to main_data (rolling, percentile)

main_data["oct_data_filtered"] = depth_5_95
#main_data = main_data.drop("oct_data", axis = 1)
#main_data

In [ ]:
#Building mean of filtered data in new column (rolling, percentile)

depth_5_95_mean = []

for idx, row in main_data.iterrows():

    depth_5_95_mean.append(row["oct_data_filtered"].depth_measured.values.mean())

main_data["oct_data_filtered_mean"] = depth_5_95_mean

## 2.3 Histogram of filtered data of id 1001.1 (rolling, percentile)

In [ ]:
#Histogram of filtered data of id 1001.1 (rolling, percentile)

h = sorted(main_data.iloc[0].oct_data_filtered.depth_measured.values)  #sorted

fit = stats.norm.pdf(h, np.mean(h), np.std(h))

pl.plot(h,fit,'-')

pl.hist(h,density=True)

pl.axis( [0, 1500, 0, 0.004] )
pl.xlabel ("gemessene Tiefe in [µm]")
pl.ylabel ("Wahrscheinlichkeitsdichte")
pl.title ('id 1001.1 aus filtered data')

#plt.savefig(str(rolling_val)+"_id 1001.1 aus filtered data"+'_' +str(ts_size)+"_"+ Test+ plot_format, dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

pl.show()

## 2.4 Plots for filtered data (rolling, percentile)

In [ ]:
#Plots for filtered data (rolling, percentile)

#for i in range(0,10): #Enter range between 0 and 630
#
#    plt.figure(i) #Open new figure
#    plt.scatter(main_data.iloc[i].oct_data_filtered.index, main_data.iloc[i].oct_data_filtered.depth_measured.values, color = 'b', marker = '.', label = 'Messpunkte') #Plot measurements
#    plt.hlines(main_data.iloc[i].depth,0, main_data.iloc[i].oct_data_filtered.index[-1], color = 'r', label = 'Einschweißtiefe') #Plot horizontal line for weld depth
#    plt.hlines(main_data.iloc[i].oct_data_filtered.depth_measured.values.mean(),0, main_data.iloc[i].oct_data_filtered.index[-1], color = 'y', label = 'Mittelwert Messpunkte') #Plot horizontal line for mean of measured depth by oct
#    plt.plot(main_data.iloc[i].oct_data_filtered.depth_measured.rolling(500).mean(), color = 'g',label = 'Messungen gemittelt über 500 Messpunkte') #Plot mean value for every 500 points
#    plt.title('id ' + str(main_data.index[i]))
#    plt.ylabel ("Tiefe [µm]")
#    plt.xlabel ("Zeit in [s]")
#    plt.legend()

## 2.5 Filtering data additionally to standard deviation (rolling, percentile, std)

In [ ]:
#Filtering data additionally to standard deviation (rolling, percentile, std) (does not really improve result)

oct_depth_std = []

for idx, row in main_data.iterrows():
    depth_std = np.std(row.oct_data_filtered.depth_measured) #Calc standard deviation
    oct_depth_std.append(row["oct_data_filtered"][lambda x: x.depth_measured.between(row.oct_data_filtered_mean-(depth_std*1), row.oct_data_filtered_mean+(depth_std*1))]) #Select data between mean-std*x and mean+std*x, x can be adapted

In [ ]:
#Adding filtered data 2 to main_data (rolling, percentile, std)

main_data["oct_data_filtered_2"] = oct_depth_std

In [ ]:
#Building mean of filtered data 2 in new column (rolling, percentile, std)

depth_mean_2 = []

for idx, row in main_data.iterrows():

    depth_mean_2.append(row["oct_data_filtered_2"].depth_measured.values.mean())

main_data["oct_data_filtered_2_mean"] = depth_mean_2

In [ ]:
#Building median of filtered data 2 in new column (rolling, percentile, std)

depth_median_2 = []

for idx, row in main_data.iterrows():

    depth_median_2.append(statistics.median(row["oct_data_filtered_2"].depth_measured.values))

main_data["oct_data_filtered_2_median"] = depth_median_2

In [ ]:
#Histogram of filtered data of id 1001.1 (rolling, percentile)

h = sorted(main_data.iloc[0].oct_data_filtered_2.depth_measured.values)  #sorted

fit = stats.norm.pdf(h, np.mean(h), np.std(h))

pl.plot(h,fit,'-')

pl.hist(h,density=True)

pl.axis( [0, 1500, 0, 0.004] )
pl.xlabel ("gemessene Tiefe in [µm]")
pl.ylabel ("Wahrscheinlichkeitsdichte")
pl.title ('id 1001.1 aus filtered data 2')

#plt.savefig(str(rolling_val)+"_id 1001.1 aus filtered data 2"+'_' +str(ts_size)+"_"+ Test+ plot_format, dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

pl.show()

In [ ]:
#Plots for filtered data 2 (rolling, percentile, std)

#for i in range(0,10):

#    plt.figure(i) #Open new figure
#    plt.scatter(main_data.iloc[i].oct_data_filtered_2.index, main_data.iloc[i].oct_data_filtered_2.depth_measured.values, color = 'b', marker = '.', label = 'Messpunkte') #Plot measurements
#    plt.hlines(main_data.iloc[i].depth,0, main_data.iloc[i].oct_data_filtered_2.index[-1], color = 'r', label = 'Einschweißtiefe') #Plot horizontal line for weld depth
#    plt.hlines(main_data.iloc[i].oct_data_filtered_2.depth_measured.values.mean(),0, main_data.iloc[i].oct_data_filtered_2.index[-1], color = 'y', label = 'Mittelwert Messpunkte') #Plot horizontal line for mean of measured depth by oct
#    plt.plot(main_data.iloc[i].oct_data_filtered_2.depth_measured.rolling(500).mean(), color = 'g',label = 'Messungen gemittelt über 500 Messpunkte') #Plot mean value for every 500 points
#    plt.title('id ' + str(main_data.index[i]))
#    plt.ylabel ("Tiefe [µm]")
#    plt.xlabel ("Zeit in [s]")
#    plt.legend()

# 3. Modelling

Lineare Regressionsmodelle:
    - analytical OLS
    - maximum likelihood (The maximum likelihood approach builds on the assumption that there is one fixed set of parameters maximizing the likelihood function. -> we don't need it)
      - Bayesian regression (to do:Contrary to the maximum likelihood approach, a fully Bayesian approach considers all plausible parameters and not just one single set of parameters.)
      
Nichtlineare Regressionsmodell:
    - polynomial regression (ab pol grad 4 gibt es ein Overfitting -> schlechteres Vorhersagen)
    
Overfitting & Regularization: (unser R^2 ist so good, dass wir keine Ridge/Lasso (penalty term) brauchen))
    - Ridge regression
    - Lasso regression
  
    
Cross validation:
    - K-fold-cross-validation ( als Ausblick gedacht -> einfachere Filtermethode für Raw-Date)
    

## 3.1 Linear regression model for all id's

### Apply OLS linear regression with scikit-learn

In [ ]:
X = main_data[['power', 'speed', 'angle', 'oct_data_filtered_2_mean']] #Select between oct_data_filtered_mean, oct_data_filtered_2_mean and oct_data_filtered_2_median, take input variables
y = main_data['depth'] #take output variable

In [ ]:
#Train_test_split

X_train_ols, X_test_ols, y_train_ols, y_test_ols = train_test_split(X, y, test_size=ts_size) #train_test_split randomly splits the data


### Scale data

In [ ]:
scaler.fit(X_train_ols) #scaler.fit() to training data
X_scaled_train_ols = scaler.transform(X_train_ols) #Scale training data X_train to mean zero and standard deviation 1


In [ ]:
#Test whether data is correctly scaled

print('Mean: %.2f'%np.mean(X_scaled_train_ols))
print('Standard deviation: %.2f'%np.std(X_scaled_train_ols))
if (np.isclose(np.mean(X_scaled_train_ols),0)) and (np.isclose(np.std(X_scaled_train_ols),1)): # we use np.isclose() function
    print('Training data successfuly scaled!')

### Fit linear regression model

In [ ]:
regressor = LinearRegression(normalize=False, fit_intercept=True) #Instantiate an object of the Linear Regression class

regressor.fit(X_scaled_train_ols, y_train_ols) #Train the instance (regressor) using the .fit() function with the scaled data (X_scaled_train)

In [ ]:
X_scaled_test_ols = scaler.transform(X_test_ols) #Scaler transforms test data (X_test)

y_hat_ols = regressor.predict(X_scaled_test_ols) #.predict() function predicts y_hat values based on x_scaled_test

plt.rcParams["figure.figsize"] = (8,6)

plt.figsize=(8,6)

plt.scatter(X_test_ols['oct_data_filtered_2_mean'], y_test_ols, s=10, alpha=0.3,label = "Reale Einschweißtiefe")

plt.plot(X_test_ols['oct_data_filtered_2_mean'],y_hat_ols,'o', label="Grad %d" % 1

             +'; $R^2$: %.3f' % regressor.score(X_scaled_test_ols, y_test_ols) + ', MSE: %.2f'%mse(y_test_ols, y_hat_ols))

plt.legend(loc=2)

plt.xlabel("Mittelwerte der OCT-Messungen in [µm]")

plt.ylabel("Vorausgesagte und reale Einschweißtiefe in [µm]")

plt.title(plot_title_3)

pl.axis( [500, 2000, 500, 2500] )

plt.savefig(str(rolling_val)+"_"+plot_title_3+'_' +str(ts_size)+"_"+Test+ plot_format,dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
print('Mean squared error OLS: %.3f'%mse(y_test_ols, y_hat_ols)) # from utils; Ergebnis in %

## 3.2 Linear regression model for id 1101.1 to 1110.1

In [ ]:
X_1 = main_data.iloc[10:20][['power', 'speed', 'angle', 'oct_data_filtered_2_mean']] # take the data set and drop (i.e. delete) the output variables
y_1 = main_data.iloc[10:20]['depth'] # take the output variable of interest and define as y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.20) #train_test_split randomly splits the data

In [ ]:
scaler.fit(X_train)#scaler.fit() to training data x; wichtig: nicht auf Testdaten

X_scaled_train = scaler.transform(X_train)#Scale training data X_train to mean zero and standrd deviation 1

In [ ]:
regressor = LinearRegression(normalize=False, fit_intercept=True) #Instantiate an object of the Linear Regression class

regressor.fit(X_scaled_train, y_train) #Train the instance (regressor) using the .fit() function with the scaled data (X_scaled_train)

In [ ]:
X_scaled_test = scaler.transform(X_test) #ToDo: Use scaler to transform teh test data (X_test)

y_hat = regressor.predict(X_scaled_test) #ToDo: Use the .predict() function to predict y_hat values based on x_scaled_test with the regresy_hat

In [ ]:
print('Mean squared error: %.2f'%mse(y_test, y_hat)) # from utils; Ergebnis in %

## 3.3 Polynomial features

In [ ]:
#Train_test_split
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X, y, test_size=ts_size) #train_test_split randomly splits the data

In [ ]:
#Plot polynom

plt.rcParams["figure.figsize"] = (8,6)

plt.scatter(X_test_poly['oct_data_filtered_2_mean'], y_test_poly, s=10, alpha=0.3,label ="Reale Einschweißtiefe")


best_hat_poly =[]

best_R_val = []

plt.figsize=(8,6)

for degree in range(1,5):
    pipeline_Poly_Features = Pipeline(steps = [
                                    ('transformer', PolynomialFeatures(degree)),
                                    ('scaler', scaler),
                                    ('model', LinearRegression(normalize=True)),
])

    pipeline_Poly_Features.fit(X_train_poly, y_train_poly) # fit pipeline. In this case standard scaler and linear regression model

    y_hat_poly = pipeline_Poly_Features.predict(X_test_poly) # predict using pipeline object

    best_hat_poly.append(y_hat_poly)

    plt.plot(X_test_poly['oct_data_filtered_2_mean'], y_hat_poly,'o', label="Grad %d" % degree


             +'; $R^2$: %.3f' % pipeline_Poly_Features.score(X_test_poly, y_test_poly) + ', MSE: %.2f'%mse(y_test_poly, y_hat_poly))
    best_R_val.append(pipeline_Poly_Features.score(X_test_poly, y_test_poly))

plt.legend(loc=2)

plt.xlabel("Mittelwerte der OCT-Messungen in [µm]")

plt.ylabel("Vorausgesagte und reale Einschweißtiefe in [µm]")

plt.title(plot_title_1)

pl.axis( [500, 2000, 500, 2500] )

plt.savefig(str(rolling_val)+"_"+plot_title_1+'_' +str(ts_size)+"_"+Test+ plot_format,dpi=800, facecolor='w', edgecolor='w',orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

plt.show()

index=best_R_val.index(max(best_R_val))

y_hat_poly=best_hat_poly[index]

In [ ]:
print('Mean squared error: %.2f'%mse(y_test_poly, y_hat_poly)) # from utils; Ergebnis in %

## 3.4 Bayesian Ridge Regression

In [ ]:
reg = linear_model.BayesianRidge(compute_score=True,normalize=True,alpha_init =1,lambda_init=1,n_iter = 1000)
#training
X_train_bayes, X_test_bayes, y_train_bayes, y_test_bayes = train_test_split(X, y, test_size=ts_size)

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)

reg.fit(X_train_bayes, y_train_bayes)

y_hat_bayes = reg.predict(X_test_bayes)

plt.figsize=(8,6)

plt.scatter(X_test_bayes['oct_data_filtered_2_mean'], y_test_bayes, s=10, alpha=0.3,label ="Reale Einschweißtiefe")

plt.plot(X_test_bayes['oct_data_filtered_2_mean'], y_hat_bayes,'o',label= '$R^2$: %.3f' %reg.score(X_test_bayes, y_test_bayes)+ ', MSE: %.2f'%mse(y_test_bayes, y_hat_bayes))

plt.legend(loc=2)

plt.xlabel("Mittelwerte der OCT-Messungen in [µm]")

plt.ylabel("Vorausgesagte und reale Einschweißtiefe in [µm]")

plt.title(plot_title_2)

pl.axis( [500, 2000, 500, 2500] )

plt.savefig(str(rolling_val)+"_"+plot_title_2+'_' +str(ts_size)+"_"+Test+ plot_format, dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

plt.show()


In [ ]:
print('Mean squared error Bayesian: %.3f'%mse(y_test_bayes, y_hat_bayes))

# 3.4 Results and Model Insights

In [ ]:
#OLS Plot
plotHist(y_test_ols,y_hat_ols,"Standard-skalierte Schweißnahttiefe","Wahrscheinlichkeitsdichte", "OLS-Regression:","Gauss_ols",ts_size,plot_format)

In [ ]:
#Poly Plot
plotHist(y_test_poly,y_hat_poly,"Standard-skalierte Schweißnahttiefe","Wahrscheinlichkeitsdichte", "Polynom-Regression:","Gauss_Poly",ts_size,plot_format)

In [ ]:
#Bayes Plot
plotHist(y_test_bayes,y_hat_bayes,"Standard-skalierte Schweißnahttiefe","Wahrscheinlichkeitsdichte", "Bayesian-Regression:","Gaus_Bayes",ts_size,plot_format)

In [ ]:
#Plot y_test data in blue and y_hat in red

y_test_array_ols = np.array(y_test_ols) #change type to array

y_hat_array_ols = np.array(y_hat_ols) #change type to array

x = np.linspace(0, len(y_test_ols)-1, len(y_test_ols))

for i in range (0,len(y_test_ols)-1):

    plt.scatter(x[i], y_test_array_ols[i], color = 'b', marker = '.')

    plt.scatter(x[i], y_hat_array_ols[i], color = 'r', marker = '.')

plt.title("OLS: Vergleich zwischen vorhergesagter und realer Tiefe") # linear regression

plt.legend(('Reale Tiefe', 'Vorhergesagte Tiefe'),loc=2)

plt.xlabel("Nummer der Test-Messung")

plt.ylabel("Vorhergesagte und reale Tiefe in [µm]")

pl.axis( [0, x[i]+5, 0, 2500] )

#plt.savefig(str(rolling_val)+"_"+'OLS_VGL'+'_' +str(ts_size)+"_"+ Test+plot_format, dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
y_test_array_poly = np.array(y_test_poly) #change type to array
y_hat_array_poly = np.array(y_hat_poly) #change type to array


x = np.linspace(0, len(y_test_poly)-1, len(y_test_poly))

for i in range (0,len(y_test_poly)-1):

    plt.scatter(x[i], y_test_array_poly[i], color = 'b', marker = '.')
    plt.scatter(x[i], y_hat_array_poly[i], color = 'r', marker = '.')

plt.title("Polynom: Vergleich zwischen vorhergesagter und realer Tiefe") # linear regression

plt.legend(('Reale Tiefe', 'Vorhergesagte Tiefe'),loc=2)

plt.xlabel("Nummer der Test-Messung")

plt.ylabel("Vorhergesagte und reale Tiefe in [µm]")

pl.axis( [0, x[i]+5, 0, 2500] )

#plt.savefig(str(rolling_val)+"_"'Poly_VGL'+'_' +str(ts_size)+"_"+ Test+plot_format, dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
y_test_array_bayes = np.array(y_test_bayes) #change type to array
y_hat_array_bayes = np.array(y_hat_bayes) #change type to array
x = np.linspace(0, len(y_test_bayes)-1, len(y_test_bayes))

for i in range (0,len(y_test_bayes)-1):

    plt.scatter(x[i], y_test_array_bayes[i], color = 'b', marker = '.')
    plt.scatter(x[i], y_hat_array_bayes[i], color = 'r', marker = '.')

plt.title("Bayesian: Vergleich zwischen vorhergesagter und realer Tiefe") # linear regression

plt.legend(('Reale Tiefe', 'Vorhergesagte Tiefe'),loc=2)

plt.xlabel("Nummer der Test-Messung")

plt.ylabel("Vorhergesagte und reale Tiefe in [µm]")

pl.axis( [0, x[i]+5, 0, 2500] )

#plt.savefig(str(rolling_val)+"_"+'Bayes_VGL'+'_' +str(ts_size)+"_"+ Test+ plot_format, dpi=800, facecolor='w', edgecolor='w', orientation='portrait', papertype='a5',transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
#https://www.studytonight.com/post/what-is-mean-squared-error-mean-absolute-error-root-mean-squared-error-and-r-squared

print('Mean squared error MSE von OLS: %.3f'%mse(y_test_ols, y_hat_ols)) # from utils; Ergebnis in %

print('Root mean square RMS von OLS: %.3f'%rms(y_test_ols, y_hat_ols)) # from utils; Ergebnis in %

print('Mean absolute error MAE von OLS: %.3f'%mae(y_test_ols, y_hat_ols)) # from utils; Ergebnis in %

print('Coefficient of determination R^2 von OLS: %.3f' %r2_score(y_test_ols, y_hat_ols))

In [ ]:
print('Mean squared error MSE von Polynom: %.3f'%mse(y_test_poly, y_hat_poly)) # from utils; Ergebnis in %

print('Root mean square RMS von Polynom: %.3f'%rms(y_test_poly, y_hat_poly)) # from utils; Ergebnis in %

print('Mean absolute error MAE von Polynom: %.3f'%mae(y_test_poly, y_hat_poly)) # from utils; Ergebnis in %

print('Coefficient of determination R^2 von Polynom: %.3f' %r2_score(y_test_poly, y_hat_poly))

In [ ]:
print('Mean squared error MSE von Bayesian: %.3f'%mse(y_test_bayes, y_hat_bayes)) # from utils; Ergebnis in %

print('Root mean square RMS von Bayesian: %.3f'%rms(y_test_bayes, y_hat_bayes)) # from utils; Ergebnis in %

print('Mean absolute error MAE von Bayesian: %.3f'%mae(y_test_bayes, y_hat_bayes)) # from utils; Ergebnis in %

print('Coefficient of determination R^2 von Bayesian: %.3f' %r2_score(y_test_bayes, y_hat_bayes))